In [104]:
import numpy as np
import os
import cv2
import pandas as pd
from tqdm import tqdm

In [34]:

# Directory containing the npz files
directory = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal/dump_match_pairs'

# List all files in the directory
files = os.listdir(directory)

# Filter out the npz files
npz_files = [file for file in files if file.endswith('.npz')]


In [67]:
import matplotlib.pyplot as plt

def visualize_matches(image1, image2, kpts1, kpts2, matches, mask):
    """
    Visualize the matches between two images with inliers and outliers.

    Parameters:
    - image1: The first image.
    - image2: The second image.
    - kpts1: Keypoints in the first image.
    - kpts2: Keypoints in the second image.
    - matches: Array of matches.
    - mask: Mask indicating inliers (1) and outliers (0).
    """
    # Convert keypoints to integer
    kpts1 = np.int32(kpts1)
    kpts2 = np.int32(kpts2)

    # Create a new image by concatenating the two images side by side
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    new_image = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    new_image[:h1, :w1] = image1
    new_image[:h2, w1:w1 + w2] = image2

    # Draw matches
    for i, match in enumerate(matches):
        color = (0, 255, 0) if mask[i] else (0, 0, 255)  # Green for inliers, Red for outliers
        pt1 = (kpts1[matches[match]][0], kpts1[matches[match]][1])
        pt2 = (kpts2[matches[match]][0], kpts2[matches[match]][1])
        cv2.line(new_image, pt1, pt2, color, 1)
        cv2.circle(new_image, pt1, 5, color, -1)
        cv2.circle(new_image, pt2, 5, color, -1)

    # Display the image
    plt.figure(figsize=(15, 10))
    plt.imshow(new_image)
    plt.axis('off')
    plt.show()


In [35]:
len(npz_files)

18840

In [69]:
fillll = npz_files[8]
print(fillll)
data = np.load(os.path.join(directory, fillll))

84569016_10080958105-05894470_4949089886-matches.npz


In [16]:
matches = data['matches']
kpts0 = data['keypoints0']
kpts1 = data['keypoints1']
valconf = data['match_confidence']

In [17]:
valid = matches > -1
mkpts0 = kpts0[valid]
mkpts1 = kpts1[matches[valid]]
valconf = valconf[valid]

In [18]:
len(valconf)


147

In [20]:
THRESH = 0.7
count = np.sum(valconf > THRESH)
print(count)

30


In [122]:
def predict_for_pair(npz_file, est_max_iter=10000):
    # Load the npz file
    data = np.load(os.path.join(directory, npz_file))
    exception = None

    # Extract matches and keypoints
    matches = data['matches']
    kpts0 = data['keypoints0']
    kpts1 = data['keypoints1']
    match_conf = data['match_confidence']
    
    # Filter valid matches
    valid = []
    cnt = 0
    bad = 0
    mkpts0 = []
    mkpts1 = []
    
    AMOUNT = 30
    THRESH = 0.95
    while cnt < AMOUNT and THRESH >= 0.5:
        mkpts0 = []
        mkpts1 = []
        cnt = 0
        for i in range(len(matches)):
            is_valid = matches[i] > -1 and match_conf[i] >= THRESH
            if is_valid:
                cnt += 1
                mkpts0.append(kpts0[i])
                mkpts1.append(kpts1[matches[i]])
        THRESH -= 0.05
    
    #valid = matches > -1
    #mkpts0 = kpts0[valid]
    #mkpts1 = kpts1[matches[valid]]
    try:
    # Estimate the fundamental matrix using the matched keypoints
        #F, mask = cv2.findFundamentalMat(np.array(mkpts0), np.array(mkpts1), cv2.FM_RANSAC, confidence=0.999999)
        F, mask = cv2.findFundamentalMat(np.array(mkpts0), np.array(mkpts1), cv2.USAC_MAGSAC, ransacReprojThreshold=1.25, confidence=0.99999, maxIters=est_max_iter)
    except Exception as e:
        exception = type(e).__name__
        #print(cnt)
        #print(npz_file)
        F = None
    
    
    # Extract scene name and image names
    scene_name = data['scene_name']
    image_name0 = npz_file.split('-')[0]
    image_name1 = npz_file.split('-')[1]
    
    
    # Append the results to the lists
    sample = f"{scene_name};{image_name0}-{image_name1}"
    
    if F is not None:
        if F.shape != (3,3):
            print(F.shape)
        F = F.reshape(-1)
        F = F[:9]
        
    return sample, F, exception

In [117]:
samples = []
fundamental_matrices = []
exception_counts = {}


# Iterate over all npz files
for npz_file in tqdm(npz_files):
    sample, F, exp = predict_for_pair(npz_file, est_max_iter = 10000)
    
    samples.append(sample)
    fundamental_matrices.append(F)
    if exp in exception_counts:
            exception_counts[exp] += 1
    else:
        exception_counts[exp] = 1

# Create a dataframe with the collected data
df = pd.DataFrame({
    'sample_id': samples,
    'fundamental_matrix': fundamental_matrices
})

print(exception_counts)

100%|████████████████████████████████████████████████████████████████████████████████| 18840/18840 [01:08<00:00, 274.12it/s]

{None: 18628, 'error': 212}


In [118]:
missing = df.sample_id[df.fundamental_matrix.isna()]
missing_df = pd.DataFrame(missing, columns=['sample_id'])
print(missing_df)

                                               sample_id
25     st_pauls_cathedral;94451439_81070120-87897753_...
153    notre_dame_front_facade;86370842_5935029406-26...
217    notre_dame_front_facade;73426073_4652539077-63...
277    notre_dame_front_facade;63964458_5239892921-40...
365    notre_dame_front_facade;88420798_4886011265-30...
...                                                  ...
18542  notre_dame_front_facade;95177872_8272975292-90...
18573  notre_dame_front_facade;86370842_5935029406-05...
18582  notre_dame_front_facade;95177872_8272975292-70...
18611  trevi_fountain;43275749_5263794765-15742678_30...
18766  notre_dame_front_facade;86370842_5935029406-63...

[237 rows x 1 columns]


In [119]:

missing_df['fundamental_matrix'] = missing_df.apply(lambda row: np.random.rand(9), axis=1)
final = pd.concat([df, missing_df]).reset_index(drop=True)[['sample_id', 'fundamental_matrix']]

final = final.dropna()
final['fundamental_matrix'] = final['fundamental_matrix'].apply(lambda x: ' '.join(f'{num:.5e}' for num in x))

In [120]:
final

,sample_id,fundamental_matrix
0,notre_dame_front_facade;59331375_2244201695-53...,-6.07589e-08 6.73554e-06 -3.40953e-03 -6.56035...
1,st_peters_square;91000026_2755820233-61888375_...,1.38678e-06 -3.67398e-06 -4.09916e-05 -3.93117...
2,trevi_fountain;31673288_4705285644-22016165_10...,3.16302e-07 -7.18447e-06 3.79314e-03 8.05377e-...
3,st_peters_square;44455970_5746646894-08838274_...,-2.90849e-07 -6.48779e-07 4.99668e-03 5.17126e...
4,st_pauls_cathedral;60046937_167213677-37891131...,8.06096e-07 3.11619e-06 -2.37760e-04 1.67213e-...
...,...,...
19072,notre_dame_front_facade;95177872_8272975292-90...,5.23883e-01 1.98649e-01 1.50717e-01 4.68358e-0...
19073,notre_dame_front_facade;86370842_5935029406-05...,3.82198e-01 7.38903e-01 4.50760e-01 3.07736e-0...
19074,notre_dame_front_facade;95177872_8272975292-70...,1.50871e-01 4.70430e-01 9.89498e-01 6.88163e-0...
19075,trevi_fountain;43275749_5263794765-15742678_30...,3.03218e-01 3.80904e-01 4.33788e-01 6.07236e-0...


In [121]:
final.to_csv("/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal/submit_result_fixes.csv", index=False)

In [45]:
f = final.iloc[0]
f

sample_id             notre_dame_front_facade;59331375_2244201695-53...
fundamental_matrix    -3.03500e-07 -5.05805e-05 5.02779e-03 4.86288e...
Name: 0, dtype: object

In [70]:
def BuildCompositeImage(im1, im2, axis=1, margin=0, background=1):
    '''Convenience function to stack two images with different sizes.'''
    
    if background != 0 and background != 1:
        background = 1
    if axis != 0 and axis != 1:
        raise RuntimeError('Axis must be 0 (vertical) or 1 (horizontal')

    h1, w1, _ = im1.shape
    h2, w2, _ = im2.shape

    if axis == 1:
        composite = np.zeros((max(h1, h2), w1 + w2 + margin, 3), dtype=np.uint8) + 255 * background
        if h1 > h2:
            voff1, voff2 = 0, (h1 - h2) // 2
        else:
            voff1, voff2 = (h2 - h1) // 2, 0
        hoff1, hoff2 = 0, w1 + margin
    else:
        composite = np.zeros((h1 + h2 + margin, max(w1, w2), 3), dtype=np.uint8) + 255 * background
        if w1 > w2:
            hoff1, hoff2 = 0, (w1 - w2) // 2
        else:
            hoff1, hoff2 = (w2 - w1) // 2, 0
        voff1, voff2 = 0, h1 + margin
    composite[voff1:voff1 + h1, hoff1:hoff1 + w1, :] = im1
    composite[voff2:voff2 + h2, hoff2:hoff2 + w2, :] = im2

    return (composite, (voff1, voff2), (hoff1, hoff2))


def DrawMatches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    '''Draw keypoints and matches.'''
    
    composite, v_offset, h_offset = BuildCompositeImage(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
     

                              int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    return composite

In [74]:
row = final.iloc[0]
scene = row.sample_id.split(';')[0]
im1_name = row.sample_id.split(';')[1].split('-')[0] + '.jpg'
im2_name = row.sample_id.split(';')[1].split('-')[1] + '.jpg'

In [75]:
p1 = f"/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/test_images/{scene}/{im1_name}"
p2 = f"/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/test_images/{scene}/{im2_name}"

In [77]:
im1 = cv2.imread(p1)
im2 = cv2.imread(p2)


In [87]:
filename = f"{im1_name.split('.')[0]}-{im2_name.split('.')[0]}-matches.npz"
print(filename)

59331375_2244201695-53172752_2917751834-matches.npz


In [90]:
data = np.load(os.path.join('/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal/dump_match_pairs', filename))


In [99]:
# Load the npz file

# Extract keypoints and matches
kpts0 = data['keypoints0']
kpts1 = data['keypoints1']
matches = data['matches']
match_conf = data['match_confidence']

# Filter valid matches
valid = matches > -1

mkpts0 = kpts0[valid]
mkpts0 = [[int(k[0]),int(k[1])] for k in mkpts0]
mkpts1 = kpts1[matches[valid]]
mkpts1 = [[int(k[0]),int(k[1])] for k in mkpts1]

valconf = match_conf[valid]


In [100]:
mkpts0

[[321, 215],
 [338, 215],
 [384, 46],
 [403, 230],
 [555, 166],
 [301, 205],
 [315, 202],
 [181, 354],
 [452, 86],
 [345, 166],
 [596, 190],
 [105, 188],
 [631, 137],
 [36, 428],
 [358, 259],
 [378, 91],
 [330, 201],
 [277, 25],
 [311, 214],
 [576, 195],
 [500, 48],
 [454, 353],
 [418, 34],
 [446, 204],
 [247, 104],
 [488, 43],
 [215, 188],
 [394, 23],
 [488, 141],
 [222, 136],
 [313, 261],
 [50, 260],
 [419, 238],
 [129, 152],
 [392, 415],
 [84, 104],
 [239, 237],
 [501, 64],
 [525, 262],
 [195, 168],
 [453, 101],
 [207, 236],
 [196, 398],
 [66, 242],
 [554, 261],
 [181, 86],
 [206, 142],
 [423, 366],
 [130, 62],
 [450, 47],
 [144, 429],
 [522, 461],
 [510, 130],
 [475, 190],
 [569, 167],
 [213, 202],
 [528, 87],
 [151, 159],
 [101, 426],
 [467, 262],
 [45, 429],
 [191, 235],
 [525, 63],
 [607, 329],
 [89, 141],
 [446, 191],
 [125, 86],
 [346, 249],
 [402, 134],
 [419, 204],
 [128, 72],
 [74, 163],
 [503, 87],
 [594, 447],
 [345, 202],
 [475, 74],
 [460, 174],
 [564, 322],
 [411, 69],

In [96]:
zip(mkpts0, mkpts1)

In [94]:
matches

array([  81,    5,   -1, ...,   -1,  888, 1014], shape=(1024,))

In [101]:
DrawMatches(im1, im2, mkpts0, mkpts1, zip(mkpts0, mkpts1), axis=1, margin=0, background=0, linewidth=2)


TypeError: list indices must be integers or slices, not tuple